# Sparkify Project Workspace 
This workspace contains a tiny subset (128MB) of the full dataset available (12GB). Feel free to use this workspace to build your project, or to explore a smaller subset with Spark before deploying your cluster on the cloud. Instructions for setting up your Spark cluster is included in the last lesson of the Extracurricular Spark Course content.

You can follow the steps below to guide your data analysis and model building portion of this project.

In [111]:
# import libraries
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
from pyspark.sql.types import IntegerType
from pyspark.sql import functions as F
from pyspark.sql.functions import desc
from pyspark.sql.functions import asc, col, collect_list, collect_set
from pyspark.sql.functions import sum as Fsum

import datetime

import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt

from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.feature import CountVectorizer, IDF, Normalizer, PCA, RegexTokenizer, StandardScaler, StopWordsRemover, StringIndexer, VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder


In [2]:
# create a Spark session
spark = SparkSession \
    .builder \
    .appName("sparkify") \
    .getOrCreate()

# Load and Clean Dataset
Clean your dataset, checking for invalid or missing data. For example, records without userids or sessionids. In this workspace, the filename is `mini_sparkify_event_data.json`.

In [3]:
path = 'mini_sparkify_event_data.json'
user_log = spark.read.json(path)

In [4]:
user_log.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



In [5]:
user_log.describe()

DataFrame[summary: string, artist: string, auth: string, firstName: string, gender: string, itemInSession: string, lastName: string, length: string, level: string, location: string, method: string, page: string, registration: string, sessionId: string, song: string, status: string, ts: string, userAgent: string, userId: string]

In [6]:
user_log.describe().show()

+-------+------------------+----------+---------+------+------------------+--------+-----------------+------+-----------------+------+-------+--------------------+-----------------+--------------------+------------------+--------------------+--------------------+-----------------+
|summary|            artist|      auth|firstName|gender|     itemInSession|lastName|           length| level|         location|method|   page|        registration|        sessionId|                song|            status|                  ts|           userAgent|           userId|
+-------+------------------+----------+---------+------+------------------+--------+-----------------+------+-----------------+------+-------+--------------------+-----------------+--------------------+------------------+--------------------+--------------------+-----------------+
|  count|            228108|    286500|   278154|278154|            286500|  278154|           228108|286500|           278154|286500| 286500|            

In [7]:
user_log.take(3)

[Row(artist='Martha Tilston', auth='Logged In', firstName='Colin', gender='M', itemInSession=50, lastName='Freeman', length=277.89016, level='paid', location='Bakersfield, CA', method='PUT', page='NextSong', registration=1538173362000, sessionId=29, song='Rockpools', status=200, ts=1538352117000, userAgent='Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) Gecko/20100101 Firefox/31.0', userId='30'),
 Row(artist='Five Iron Frenzy', auth='Logged In', firstName='Micah', gender='M', itemInSession=79, lastName='Long', length=236.09424, level='free', location='Boston-Cambridge-Newton, MA-NH', method='PUT', page='NextSong', registration=1538331630000, sessionId=8, song='Canada', status=200, ts=1538352180000, userAgent='"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/37.0.2062.103 Safari/537.36"', userId='9'),
 Row(artist='Adam Lambert', auth='Logged In', firstName='Colin', gender='M', itemInSession=51, lastName='Freeman', length=282.8273, level='paid', location='

In [8]:
user_log.describe("userId").show()

+-------+-----------------+
|summary|           userId|
+-------+-----------------+
|  count|           286500|
|   mean|59682.02278593872|
| stddev|109091.9499991047|
|    min|                 |
|    max|               99|
+-------+-----------------+



In [9]:
user_log_valid = user_log.dropna(how = "any", subset = ["userId", "sessionId"])

In [10]:
user_log_valid.count()

286500

In [11]:
user_log.count()

286500

In [12]:
user_log_valid = user_log_valid.filter(user_log_valid["userId"] != "")

In [13]:
user_log_valid.count()

278154

In [14]:
user_log = user_log_valid

In [15]:
user_log.select("page").dropDuplicates().sort("page").show()

+--------------------+
|                page|
+--------------------+
|               About|
|          Add Friend|
|     Add to Playlist|
|              Cancel|
|Cancellation Conf...|
|           Downgrade|
|               Error|
|                Help|
|                Home|
|              Logout|
|            NextSong|
|         Roll Advert|
|       Save Settings|
|            Settings|
|    Submit Downgrade|
|      Submit Upgrade|
|         Thumbs Down|
|           Thumbs Up|
|             Upgrade|
+--------------------+



In [16]:
user_log.groupby(["userId"]).count().orderBy(desc("count")).show()

+------+-----+
|userId|count|
+------+-----+
|    39| 9632|
|    92| 7230|
|   140| 6880|
|300011| 5732|
|   124| 4825|
|300021| 4659|
|300017| 4428|
|    85| 4370|
|    42| 4257|
|200023| 3769|
|     6| 3761|
|    29| 3603|
|    54| 3437|
|   100| 3214|
|     9| 3191|
|   126| 3102|
|300015| 3051|
|    91| 3014|
|    98| 2891|
|    74| 2887|
+------+-----+
only showing top 20 rows



# Exploratory Data Analysis
When you're working with the full dataset, perform EDA by loading a small subset of the data and doing basic manipulations within Spark. In this workspace, you are already provided a small subset of data you can explore.

### Define Churn

Once you've done some preliminary analysis, create a column `Churn` to use as the label for your model. I suggest using the `Cancellation Confirmation` events to define your churn, which happen for both paid and free users. As a bonus task, you can also look into the `Downgrade` events.

### Explore Data
Once you've defined churn, perform some exploratory data analysis to observe the behavior for users who stayed vs users who churned. You can start by exploring aggregates on these two groups of users, observing how much of a specific action they experienced per a certain time unit or number of songs played.

In [17]:
# Select the user that has "Cancellation Confirmation" event
churn_users = user_log.filter(user_log.page == "Cancellation Confirmation").select(["userId"]).dropDuplicates(["userId"])
num_churn_users = churn_users.count()
print(num_churn_users)
churn_users.collect()

52


[Row(userId='125'),
 Row(userId='51'),
 Row(userId='54'),
 Row(userId='100014'),
 Row(userId='101'),
 Row(userId='29'),
 Row(userId='100021'),
 Row(userId='87'),
 Row(userId='73'),
 Row(userId='3'),
 Row(userId='28'),
 Row(userId='100022'),
 Row(userId='100025'),
 Row(userId='300007'),
 Row(userId='100006'),
 Row(userId='18'),
 Row(userId='70'),
 Row(userId='100005'),
 Row(userId='17'),
 Row(userId='100007'),
 Row(userId='300001'),
 Row(userId='100009'),
 Row(userId='100015'),
 Row(userId='200024'),
 Row(userId='100003'),
 Row(userId='103'),
 Row(userId='100024'),
 Row(userId='53'),
 Row(userId='122'),
 Row(userId='200017'),
 Row(userId='58'),
 Row(userId='100011'),
 Row(userId='100019'),
 Row(userId='100012'),
 Row(userId='200018'),
 Row(userId='200016'),
 Row(userId='200020'),
 Row(userId='106'),
 Row(userId='143'),
 Row(userId='32'),
 Row(userId='200001'),
 Row(userId='105'),
 Row(userId='200011'),
 Row(userId='100023'),
 Row(userId='100013'),
 Row(userId='100017'),
 Row(userId='121

In [18]:
all_users = user_log.select(["userId"]).dropDuplicates(["userId"])
num_all_users = all_users.count()
print(num_all_users)

225


In [19]:
stay_users = all_users.subtract(churn_users)
num_stay_users = stay_users.count()
print(stay_users)
stay_users.collect()

DataFrame[userId: string]


[Row(userId='100010'),
 Row(userId='200002'),
 Row(userId='124'),
 Row(userId='7'),
 Row(userId='15'),
 Row(userId='155'),
 Row(userId='132'),
 Row(userId='154'),
 Row(userId='11'),
 Row(userId='138'),
 Row(userId='300017'),
 Row(userId='69'),
 Row(userId='112'),
 Row(userId='42'),
 Row(userId='200010'),
 Row(userId='64'),
 Row(userId='113'),
 Row(userId='30'),
 Row(userId='34'),
 Row(userId='133'),
 Row(userId='59'),
 Row(userId='139'),
 Row(userId='146'),
 Row(userId='8'),
 Row(userId='22'),
 Row(userId='300018'),
 Row(userId='300012'),
 Row(userId='85'),
 Row(userId='16'),
 Row(userId='35'),
 Row(userId='52'),
 Row(userId='300008'),
 Row(userId='71'),
 Row(userId='300013'),
 Row(userId='98'),
 Row(userId='100008'),
 Row(userId='47'),
 Row(userId='99'),
 Row(userId='107'),
 Row(userId='110'),
 Row(userId='300006'),
 Row(userId='300002'),
 Row(userId='96'),
 Row(userId='43'),
 Row(userId='5'),
 Row(userId='100'),
 Row(userId='27'),
 Row(userId='61'),
 Row(userId='75'),
 Row(userId='12

In [20]:
user_log.filter((user_log.page == "Cancellation Confirmation") | (user_log.page == "Downgrade")).dropDuplicates(['userId']).count()

171

In [21]:
user_log.groupby([user_log.page]).count().show()

+--------------------+------+
|                page| count|
+--------------------+------+
|              Cancel|    52|
|    Submit Downgrade|    63|
|         Thumbs Down|  2546|
|                Home| 10082|
|           Downgrade|  2055|
|         Roll Advert|  3933|
|              Logout|  3226|
|       Save Settings|   310|
|Cancellation Conf...|    52|
|               About|   495|
|            Settings|  1514|
|     Add to Playlist|  6526|
|          Add Friend|  4277|
|            NextSong|228108|
|           Thumbs Up| 12551|
|                Help|  1454|
|             Upgrade|   499|
|               Error|   252|
|      Submit Upgrade|   159|
+--------------------+------+



## Explore behaviours among 2 different groups
You can start by exploring aggregates on these two groups of users, observing how much of a specific action they experienced per a certain time unit or number of songs played.

In [22]:
#  how much of a specific action they experienced per day
# Next Song
get_hour = udf(lambda x: datetime.datetime.fromtimestamp(x / 1000.0).hour)
get_day = udf(lambda x: datetime.datetime.fromtimestamp(x / 1000.0).day)
get_month = udf(lambda x: datetime.datetime.fromtimestamp(x / 1000.0).month)
get_year = udf(lambda x: datetime.datetime.fromtimestamp(x / 1000.0).year)


In [23]:
user_log = user_log.withColumn("hour", get_hour(user_log.ts))
user_log = user_log.withColumn("day", get_day(user_log.ts))
user_log = user_log.withColumn("month", get_month(user_log.ts))
user_log = user_log.withColumn("year", get_year(user_log.ts))

In [24]:
user_log.head()

Row(artist='Martha Tilston', auth='Logged In', firstName='Colin', gender='M', itemInSession=50, lastName='Freeman', length=277.89016, level='paid', location='Bakersfield, CA', method='PUT', page='NextSong', registration=1538173362000, sessionId=29, song='Rockpools', status=200, ts=1538352117000, userAgent='Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) Gecko/20100101 Firefox/31.0', userId='30', hour='0', day='1', month='10', year='2018')

In [25]:
churn_user_ids = [row.userId for row in churn_users.collect()]
stay_user_ids = [row.userId for row in stay_users.collect()]
print(churn_user_ids[:5])
print(stay_user_ids[:5])

['125', '51', '54', '100014', '101']
['100010', '200002', '124', '7', '15']


In [26]:
churn_user_log = user_log.where(col("userId").isin(churn_user_ids))

In [27]:
stay_user_log = user_log.where(col("userId").isin(stay_user_ids))

In [28]:
churn_user_log.groupby(["userId"]).count().orderBy(desc("count")).show()

+------+-----+
|userId|count|
+------+-----+
|    29| 3603|
|    54| 3437|
|    51| 2464|
|300001| 2168|
|   101| 2149|
|    53| 2064|
|    58| 2027|
|    28| 1781|
|    70| 1775|
|200021| 1502|
|200020| 1498|
|100013| 1392|
|   103| 1310|
|    17| 1102|
|    12| 1064|
|100015| 1050|
|   106| 1010|
|    87| 1003|
|100022|  964|
|   105|  918|
+------+-----+
only showing top 20 rows



In [29]:
stay_user_log.groupby(["userId"]).count().orderBy(desc("count")).show()

+------+-----+
|userId|count|
+------+-----+
|    39| 9632|
|    92| 7230|
|   140| 6880|
|300011| 5732|
|   124| 4825|
|300021| 4659|
|300017| 4428|
|    85| 4370|
|    42| 4257|
|200023| 3769|
|     6| 3761|
|   100| 3214|
|     9| 3191|
|   126| 3102|
|300015| 3051|
|    91| 3014|
|    98| 2891|
|    74| 2887|
|   118| 2868|
|   136| 2567|
+------+-----+
only showing top 20 rows



In [30]:
churn_user_log.groupby(["userId"]) \
    .agg({'userId':'count'}) \
    .agg({'count(userId)':'avg'}) \
    .show()

+------------------+
|avg(count(userId))|
+------------------+
| 862.7692307692307|
+------------------+



In [31]:
stay_user_log.groupby(["userId"]) \
    .agg({'userId':'count'}) \
    .agg({'count(userId)':'avg'}) \
    .show()

+------------------+
|avg(count(userId))|
+------------------+
|1348.4971098265896|
+------------------+



## Explore time information

In [32]:
user_log.select(["year", "month"]).groupby(["year", "month"]).count().show()

+----+-----+------+
|year|month| count|
+----+-----+------+
|2018|   10|144916|
|2018|   12|     4|
|2018|   11|133234|
+----+-----+------+



## Explore user profile information

In [33]:
user_log.groupby(["gender"]).count().show()

+------+------+
|gender| count|
+------+------+
|     F|154578|
|     M|123576|
+------+------+



In [34]:
user_log.groupby(["location"]).count().show()

+--------------------+-----+
|            location|count|
+--------------------+-----+
|     Gainesville, FL| 1229|
|Atlantic City-Ham...| 2176|
|Deltona-Daytona B...|   73|
|San Diego-Carlsba...|  754|
|Cleveland-Elyria, OH| 1392|
|Kingsport-Bristol...| 1863|
|New Haven-Milford...| 4007|
|Birmingham-Hoover...|   75|
|  Corpus Christi, TX|   11|
|         Dubuque, IA|  651|
|Las Vegas-Henders...| 2042|
|Indianapolis-Carm...|  970|
|Seattle-Tacoma-Be...|  246|
|          Albany, OR|   23|
|   Winston-Salem, NC|  819|
|     Bakersfield, CA| 1775|
|Los Angeles-Long ...|30131|
|Minneapolis-St. P...| 2134|
|San Francisco-Oak...| 2647|
|Phoenix-Mesa-Scot...| 4846|
+--------------------+-----+
only showing top 20 rows



In [35]:
user_log.groupby(["userId", "itemInSession"]).count().show()

+------+-------------+-----+
|userId|itemInSession|count|
+------+-------------+-----+
|    78|           33|    3|
|    78|           47|    4|
|    78|           78|    1|
|    87|            5|   22|
|    39|           28|   66|
|   142|           87|    8|
|    36|           37|    9|
|    33|           11|   11|
|    27|           54|    2|
|    92|           11|   74|
|    81|            8|   18|
|   113|           41|   21|
|   115|           18|   10|
|   101|           68|    6|
|   118|           46|   17|
|   121|          108|    2|
|   121|          124|    3|
|   101|          180|    3|
|    17|            9|    6|
|    17|           21|    5|
+------+-------------+-----+
only showing top 20 rows



In [36]:
user_log.groupby(["userId", "length"]).count().show()

+------+---------+-----+
|userId|   length|count|
+------+---------+-----+
|    25|223.81669|    2|
|    87|  241.162|    2|
|    25|244.00934|    1|
|   131|264.01914|    1|
|    39|214.12526|    1|
|    33|258.89914|    1|
|    15| 177.8673|    1|
|    81|358.66077|    1|
|   131|260.44036|    1|
|    29|207.20281|    1|
|   124|232.77669|    2|
|    61|464.92689|    1|
|    77|181.21098|    2|
|   124|245.26322|    1|
|   127|255.97342|    1|
|   100|246.80444|    2|
|    51|310.88281|    1|
|   105|162.45506|    1|
|   100|165.79873|    1|
|   100|147.25179|    1|
+------+---------+-----+
only showing top 20 rows



## Explore song and artist information

In [104]:
user_log.where(user_log.page != "NextSong").select(["song", "length"]).dropDuplicates(["song", "length"]).show()

+----+------+
|song|length|
+----+------+
|null|  null|
+----+------+



In [37]:
user_log.groupby(["song", "length"]).count().count()

65960

In [38]:
user_log.describe("length").show()

+-------+-----------------+
|summary|           length|
+-------+-----------------+
|  count|           228108|
|   mean|249.1171819778458|
| stddev|99.23517921058361|
|    min|          0.78322|
|    max|       3024.66567|
+-------+-----------------+



In [39]:
user_log.groupby(["song"]).count().count()

58481

In [40]:
user_log.describe("song").show()

+-------+--------------------+
|summary|                song|
+-------+--------------------+
|  count|              228108|
|   mean|            Infinity|
| stddev|                 NaN|
|    min|ÃÂg ÃÂtti Gr...|
|    max|ÃÂau hafa slopp...|
+-------+--------------------+



In [41]:
user_log.groupby(["artist"]).count().count()

17656

In [42]:
user_log.describe("artist").show()

+-------+------------------+
|summary|            artist|
+-------+------------------+
|  count|            228108|
|   mean| 551.0852017937219|
| stddev|1217.7693079161374|
|    min|               !!!|
|    max| ÃÂlafur Arnalds|
+-------+------------------+



# Feature Engineering
Once you've familiarized yourself with the data, build out the features you find promising to train your model on. To work with the full dataset, you can follow the following steps.
- Write a script to extract the necessary features from the smaller subset of data
- Ensure that your script is scalable, using the best practices discussed in Lesson 3
- Try your script on the full data set, debugging your script if necessary

If you are working in the classroom workspace, you can just extract features based on the small subset of data contained here. Be sure to transfer over this work to the larger dataset when you work on your Spark cluster.

In [126]:
train_data = None
train_data = user_log.groupby("userId") \
    .agg(F.max("gender").alias("gender"), \
        F.count("song").alias("numSongs"), \
        F.countDistinct("song").alias("numUniqueSongs"))
train_data.show()

+------+------+--------+--------------+
|userId|gender|numSongs|numUniqueSongs|
+------+------+--------+--------------+
|100010|     F|     275|           269|
|200002|     M|     387|           378|
|   125|     M|       8|             8|
|   124|     F|    4079|          3339|
|    51|     M|    2111|          1854|
|     7|     M|     150|           148|
|    15|     M|    1914|          1707|
|    54|     F|    2841|          2414|
|   155|     F|     820|           759|
|100014|     M|     257|           248|
|   132|     F|    1928|          1718|
|   154|     F|      84|            83|
|   101|     M|    1797|          1608|
|    11|     F|     647|           616|
|   138|     M|    2070|          1791|
|300017|     F|    3632|          3013|
|100021|     M|     230|           226|
|    29|     M|    3028|          2562|
|    69|     F|    1125|          1036|
|   112|     M|     215|           211|
+------+------+--------+--------------+
only showing top 20 rows



In [128]:
gender_indexer = StringIndexer(inputCol="gender", outputCol="genderValue")
train_data = gender_indexer.fit(train_data).transform(train_data)

In [129]:
train_data.show()

+------+------+--------+--------------+-----------+
|userId|gender|numSongs|numUniqueSongs|genderValue|
+------+------+--------+--------------+-----------+
|100010|     F|     275|           269|        1.0|
|200002|     M|     387|           378|        0.0|
|   125|     M|       8|             8|        0.0|
|   124|     F|    4079|          3339|        1.0|
|    51|     M|    2111|          1854|        0.0|
|     7|     M|     150|           148|        0.0|
|    15|     M|    1914|          1707|        0.0|
|    54|     F|    2841|          2414|        1.0|
|   155|     F|     820|           759|        1.0|
|100014|     M|     257|           248|        0.0|
|   132|     F|    1928|          1718|        1.0|
|   154|     F|      84|            83|        1.0|
|   101|     M|    1797|          1608|        0.0|
|    11|     F|     647|           616|        1.0|
|   138|     M|    2070|          1791|        0.0|
|300017|     F|    3632|          3013|        1.0|
|100021|    

In [130]:
# churn_user_ids = [row.userId for row in churn_users.collect()]
is_churn = udf(lambda x: x in churn_user_ids)


In [131]:
train_data = train_data.withColumn("is_churn", is_churn(train_data.userId))

In [132]:
# Label must be numeric value
indexer = StringIndexer(inputCol="is_churn", outputCol="label")
train_data = indexer.fit(train_data).transform(train_data)

In [133]:
train_data.show()

+------+------+--------+--------------+-----------+--------+-----+
|userId|gender|numSongs|numUniqueSongs|genderValue|is_churn|label|
+------+------+--------+--------------+-----------+--------+-----+
|100010|     F|     275|           269|        1.0|   false|  0.0|
|200002|     M|     387|           378|        0.0|   false|  0.0|
|   125|     M|       8|             8|        0.0|    true|  1.0|
|   124|     F|    4079|          3339|        1.0|   false|  0.0|
|    51|     M|    2111|          1854|        0.0|    true|  1.0|
|     7|     M|     150|           148|        0.0|   false|  0.0|
|    15|     M|    1914|          1707|        0.0|   false|  0.0|
|    54|     F|    2841|          2414|        1.0|    true|  1.0|
|   155|     F|     820|           759|        1.0|   false|  0.0|
|100014|     M|     257|           248|        0.0|    true|  1.0|
|   132|     F|    1928|          1718|        1.0|   false|  0.0|
|   154|     F|      84|            83|        1.0|   false|  

In [134]:
# Features must be vector
assembler = VectorAssembler(inputCols=["numSongs", "numUniqueSongs", "genderValue"], outputCol="NumFeatures")
train_data = assembler.transform(train_data)

In [135]:
train_data.show()

+------+------+--------+--------------+-----------+--------+-----+-------------------+
|userId|gender|numSongs|numUniqueSongs|genderValue|is_churn|label|        NumFeatures|
+------+------+--------+--------------+-----------+--------+-----+-------------------+
|100010|     F|     275|           269|        1.0|   false|  0.0|  [275.0,269.0,1.0]|
|200002|     M|     387|           378|        0.0|   false|  0.0|  [387.0,378.0,0.0]|
|   125|     M|       8|             8|        0.0|    true|  1.0|      [8.0,8.0,0.0]|
|   124|     F|    4079|          3339|        1.0|   false|  0.0|[4079.0,3339.0,1.0]|
|    51|     M|    2111|          1854|        0.0|    true|  1.0|[2111.0,1854.0,0.0]|
|     7|     M|     150|           148|        0.0|   false|  0.0|  [150.0,148.0,0.0]|
|    15|     M|    1914|          1707|        0.0|   false|  0.0|[1914.0,1707.0,0.0]|
|    54|     F|    2841|          2414|        1.0|    true|  1.0|[2841.0,2414.0,1.0]|
|   155|     F|     820|           759|    

In [136]:
# Scale the feature
scaler = Normalizer(inputCol="NumFeatures", outputCol="ScaledNumFeatures")
train_data = scaler.transform(train_data)


In [137]:
train_data.show()

+------+------+--------+--------------+-----------+--------+-----+-------------------+--------------------+
|userId|gender|numSongs|numUniqueSongs|genderValue|is_churn|label|        NumFeatures|   ScaledNumFeatures|
+------+------+--------+--------------+-----------+--------+-----+-------------------+--------------------+
|100010|     F|     275|           269|        1.0|   false|  0.0|  [275.0,269.0,1.0]|[0.71485985831292...|
|200002|     M|     387|           378|        0.0|   false|  0.0|  [387.0,378.0,0.0]|[0.71537617920553...|
|   125|     M|       8|             8|        0.0|    true|  1.0|      [8.0,8.0,0.0]|[0.70710678118654...|
|   124|     F|    4079|          3339|        1.0|   false|  0.0|[4079.0,3339.0,1.0]|[0.77380512359468...|
|    51|     M|    2111|          1854|        0.0|    true|  1.0|[2111.0,1854.0,0.0]|[0.75136274558187...|
|     7|     M|     150|           148|        0.0|   false|  0.0|  [150.0,148.0,0.0]|[0.71183643283233...|
|    15|     M|    1914|    

In [138]:
user_id_mapping = [row.userId for row in train_data.select("userId").collect()]
print(user_id_mapping[:20])

['100010', '200002', '125', '51', '124', '7', '54', '15', '155', '132', '154', '100014', '101', '11', '138', '300017', '29', '69', '100021', '42']


In [139]:
train_data2 = train_data.select(col("label"), col("ScaledNumFeatures").alias("features"))
train_data2.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|[0.71485985831292...|
|  0.0|[0.71537617920553...|
|  1.0|[0.70710678118654...|
|  0.0|[0.77380512359468...|
|  1.0|[0.75136274558187...|
|  0.0|[0.71183643283233...|
|  0.0|[0.74631113902086...|
|  1.0|[0.76205178904582...|
|  0.0|[0.73387604152160...|
|  1.0|[0.71959441596899...|
|  0.0|[0.74659683793534...|
|  0.0|[0.71130269739377...|
|  1.0|[0.74520877494892...|
|  0.0|[0.72424395032457...|
|  0.0|[0.75623125387918...|
|  0.0|[0.76964364927110...|
|  1.0|[0.71328203040941...|
|  1.0|[0.76340541102983...|
|  0.0|[0.73560496691859...|
|  0.0|[0.71371481950803...|
+-----+--------------------+
only showing top 20 rows



# Modeling
Split the full dataset into train, test, and validation sets. Test out several of the machine learning methods you learned. Evaluate the accuracy of the various models, tuning parameters as necessary. Determine your winning model based on test accuracy and report results on the validation set. Since the churned users are a fairly small subset, I suggest using F1 score as the metric to optimize.

In [149]:
rest, validation = train_data2.randomSplit([0.9, 0.1], seed=42)

In [150]:
lr =  LogisticRegression(maxIter=10, regParam=0.0, elasticNetParam=0)

pipeline = Pipeline(stages=[lr])

In [151]:
paramGrid = ParamGridBuilder() \
    .addGrid(lr.regParam,[0.0, 0.1]) \
    .build()


crossval = CrossValidator(estimator=pipeline,
                          estimatorParamMaps=paramGrid,
                          evaluator=BinaryClassificationEvaluator(),
                          numFolds=3)

In [152]:
cvModel = crossval.fit(rest)

In [153]:
cvModel.avgMetrics

[0.5915208350349861, 0.5897458411609355]

In [154]:
results = cvModel.transform(validation)

In [155]:
print(results.filter(results.label == results.prediction).count())
print(results.count())

9
16


In [158]:
print(results.filter(results.label == 1).count())


7


In [156]:
9 / 16.0

0.5625

In [157]:
(225 - 52) / 225.0

0.7688888888888888

# Final Steps
Clean up your code, adding comments and renaming variables to make the code easier to read and maintain. Refer to the Spark Project Overview page and Data Scientist Capstone Project Rubric to make sure you are including all components of the capstone project and meet all expectations. Remember, this includes thorough documentation in a README file in a Github repository, as well as a web app or blog post.